In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
today = datetime.today()
# Библиотеки для моделирования
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
# Стоп-слова, знаки пунктуации
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oigla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [11]:
plan = pd.read_csv('I:/Download/Датасеты/test_for_train_model.csv')
test = pd.read_csv('I:/Download/Датасеты/test.csv')

TF-IDF Vectorizer - векторизируем наш датафрейм, а затем по косинусному сходству подберем группу

In [12]:
tfidf = TfidfVectorizer(stop_words=list(stop_words))
plan['type3'] = plan['type3'].fillna('')
overview_matrix = tfidf.fit_transform(plan['type3'])
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
mapping = pd.Series(plan.index,index =plan['type2'])
print(mapping)

type2
ОНЛАЙН Красота и стиль           0
ОНЛАЙН Домоводство               1
ОНЛАЙН Домоводство               2
ОНЛАЙН Домоводство               3
ОНЛАЙН Домоводство               4
                             ...  
Информационные технологии    59008
Иностранные языки            59009
Иностранные языки            59010
Спортивные игры              59011
Информационные технологии    59012
Length: 59013, dtype: int64


In [13]:
plan.index

RangeIndex(start=0, stop=59013, step=1)

In [14]:
overview_matrix

<59013x458 sparse matrix of type '<class 'numpy.float64'>'
	with 178006 stored elements in Compressed Sparse Row format>

In [15]:
similarity_matrix

array([[1.       , 0.0271926, 0.0271926, ..., 0.       , 0.       ,
        0.       ],
       [0.0271926, 1.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       [0.0271926, 1.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

In [26]:
def recommend_plan(plan_input):
    plan_index = mapping[plan_input]
    similarity_score = list(enumerate(similarity_matrix[plan_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[0], reverse=True)
    similarity_score = similarity_score[1:10]
    plan_indices = [i[0] for i in similarity_score]
    return (plan['type1'].iloc[plan_indices])

In [27]:
print(recommend_plan('Фитнес, тренажеры'))

2087                    Танцы
2086                    Танцы
2085                    Танцы
2084               Творчество
2083              Образование
2082               Творчество
2081                Рисование
2080    Физическая активность
2079    Физическая активность
Name: type1, dtype: object


In [18]:
# результат сохраняем в таблицу test
test['уникальный номер группы'] = np.resize(list(recommend_plan('ОНЛАЙН Гимнастика')),len(test))
test.to_csv('I:/Download/Датасеты/test_result.csv')
test.head()

,уникальный номер участника,уникальный номер группы
0,101346610,801350520
1,101346612,801349601
2,101347269,801369467
3,101347321,801366648
4,101347983,801373454
